In [1]:
from pyaspeller import YandexSpeller, Word
from spellchecker import SpellChecker
from tqdm import tqdm
from datetime import datetime
import pandas as pd

In [2]:
# тестовые данные

data = pd.read_csv('data\prepr_set.csv')

In [3]:
from preprocessing_functions.nlp_functions import nlp_preprocessing
from preprocessing_functions.tf_idf_preprocessing import tf_idf

### Отключение парал-ти
Для того чтобы отключить парал-ть вычислений нужно зайти в файл nlp_functions.py, по поиску Parallel найти где используется эта опция, оставить пустые скобки в Parallel(). Чтобы влкючить и использовать все доступные потоки процессора нужно поставить там же в скобках n_jobs=-1.

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
# 1 
# проверка месяцев
txt1 = '1) Тексст 2) Кромольным 3) Смастирили небаскребаа 4) карова 5) чуства'
txt2 = 'Мы тестируем продукты в реальных магазинах, маркетинговый отчет о результатах продаж.'
txt3 = '1) Оля 2) Москва 3) Яндекс 4) c 5) В 6) я 7)'

df = pd.DataFrame(data=[txt1, txt2, txt3], columns=['text'])

In [20]:
# 2
df = data.iloc[:100]['txt'].to_frame()
df.columns = ['text']

### Препроцессинг текста

In [21]:
nlp_params = dict()

## Основные параметры
nlp_params['train'] = df # основной датасет
nlp_params['oos'] = None  # out-of-sample при наличии (необязательный параметр)
nlp_params['oot'] = None  # out-of-time при наличии (необязательный параметр)
nlp_params['text_field'] = 'text'  # название столбца с текстом

## Простой препроцессинг
nlp_params['need_del_dash'] = True  # удаление тире
nlp_params['need_lower_case'] = True  # приведение к нижнему регистру
nlp_params['need_del_number'] = False  # удаление чисел
nlp_params['need_del_in_brackets'] = False  # внутри скобок
nlp_params['need_del_eng'] = True  # английские буквы

## Поиск опечаток (spellcheker)
nlp_params['need_spellchecker'] = True  # запуск поиска опечаток [True, False]
# действия над опечатками [delete, replace, nothing(default)] / рекомендуется использовать replace - работает в разы быстрее
nlp_params['need_del_spell'] = 'replace'

## Лемматизация
nlp_params['need_lemma'] = True  # запуск лемматизации [True, False]
nlp_params['need_lru_cache'] = True  # использование декоратора для ускорения процесса лемматизации [True, False] (рекомендуется)

## Поиск сущностей (NER) | поиск сущностей следует запускать только после лемматизации слов
nlp_params['need_ner'] = True  # запуск поиска сущностей [True, False]
nlp_params['need_del_number_ner'] = 'nothing'  # действия над числами [delete, replace, nothing(default)]
nlp_params['need_del_name'] = 'replace'  # действия над именами (также)
nlp_params['need_del_org'] = 'replace'  # действия над названиями организаций (также)
nlp_params['need_del_geo'] = 'replace'  # действия над локациями (также)
nlp_params['need_del_months'] = True  # действия над месяцами (True - delete, False - nothing)

## Настройки для стопслов
nlp_params['need_del_stopwords'] = True  # действия над стопсловами [True, False]
nlp_params['new_stopwords'] = []  # новый список стопслов ['word1', 'word2']
# режим работы со стопсловами, какие стопслова используются ['default list', 'default list + additional list', 'only additional list']
nlp_params['mode_stopwords'] = 'default list'  

train, oos, oot = nlp_preprocessing(**nlp_params)

Trash chars done! - 0:00:00.012001



Lemmatization / Spellcheker / NER done! - 0:00:49.942236

Delete stop words done! - 0:00:00.025996
Preprocessing done!


In [23]:
train.iloc[0].to_list()

['1) Тексст 2) Кромольным 3) Смастирили небаскребаа 4) карова 5) чуства',
 '1 текст 2 крамольный 3 смастерить небоскреб 4 корова 5 чувство']

In [16]:
train.iloc[1].to_list()

['Мы тестируем продукты в реальных магазинах, маркетинговый отчет о результатах продаж.',
 ' тестировать продукт реальный магазин маркетинговый отчет результат продажа']

In [17]:
train.iloc[2].to_list()

['1) Оля 2) Москва 3) Яндекс 4) c 5) В 6) я 7)',
 '1 NAME 2 GEO 3 ORGANIZATION 4 5 6 7']

### Векторизация текста с помощью TFiDF

In [18]:
tf_idf_params = dict()

params = { # Параметры TFidfVectorizer()
    'ngram_range': (1, 1) # задаем размер н-грамм
}

## Основные параметры
tf_idf_params['train'] = train # основной датасет
tf_idf_params['oos'] = None  # out-of-sample при наличии (необязательный параметр)
tf_idf_params['oot'] = None  # out-of-time при наличии (необязательный параметр)
tf_idf_params['text_field'] = 'new_prep_text'  # название столбца с текстом
tf_idf_params['target_name'] = None  # название поля с таргетом

## Параметры TFiDF
tf_idf_params['params'] = params

train, oos, oot = tf_idf(**tf_idf_params)

In [19]:
train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.00000,0.00000,0.00000,0.408248,0.408248,0.000000,0.000000,0.408248,0.000000,0.000000,0.000000,0.000000,0.000000,0.408248,0.408248,0.000000,0.408248
1,0.00000,0.00000,0.00000,0.000000,0.000000,0.353553,0.353553,0.000000,0.353553,0.353553,0.353553,0.353553,0.353553,0.000000,0.000000,0.353553,0.000000
2,0.57735,0.57735,0.57735,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
